In [1]:
#path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'

path = r'X:\Processed_data'

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, IterableDataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch.nn as nn

window_size = 30

# Define the number of features (i.e., columns in your dataset)
#num_features = 11*window_size

num_features = 330

class AutoEncoder(nn.Module):
    def __init__(self, hidden_units1, hidden_units2, hidden_units3, latent_features):
        super(AutoEncoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=hidden_units1, kernel_size=3, stride=2, padding=1),  # Layer 1
            nn.ReLU(),
            nn.Conv2d(hidden_units1, hidden_units2, kernel_size=3, stride=2, padding=1),              # Layer 2
            nn.ReLU(),
            nn.Conv2d(hidden_units2, hidden_units3, kernel_size=3, stride=2, padding=1),              # Layer 3
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(hidden_units3 * 4 * 2, latent_features)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_features, hidden_units3 * 4 * 2),
            nn.ReLU(),
            nn.Unflatten(1, (hidden_units3, 4, 2)),
            nn.Conv2d(hidden_units3, hidden_units2, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Conv2d(hidden_units2, hidden_units1, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units1, 1, kernel_size=3, stride=2, padding=1),  
            nn.Sigmoid()
        )

    def forward(self, x): 
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return {'z': z, 'x_hat': x_hat}

# Initialize the autoencoder model
hidden_units1 = 256
hidden_units2 = 128
hidden_units3 = 64
latent_features = 32
net = AutoEncoder(hidden_units1, hidden_units2, hidden_units3, latent_features)
cuda = torch.cuda.is_available()

if cuda:
    net = net.cuda()

print(net)

## Only put batches onto gpu

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
import pyarrow.parquet as pq
import numpy as np

class FragmentDataset(Dataset):
    def __init__(self, data):
        self.data = data  # Data is already a GPU tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        x = x.view(1, 30, 11)  # Reshape to (1, 30, 11)
        return x, x  # Input and target for autoencoder

# Directory where the separate Parquet files are stored
fragments_dir = r'X:\Processed_data'

# Model, optimizer, and loss function
num_epochs = 10
batch_size = 512
model = AutoEncoder(hidden_units1, hidden_units2, latent_features)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
loss_function = torch.nn.MSELoss()

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)  # Move model to the device

# Iterate through the fragment files
fragment_files = [os.path.join(fragments_dir, f) for f in os.listdir(fragments_dir) if f.endswith('.parquet')]

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = []

    for file_path in fragment_files:
        print(f"Processing fragment: {file_path}")

        parquet_file = pq.ParquetFile(file_path)
        data = parquet_file.read().to_pandas().to_numpy(dtype='float32')

        fragment_dataset = FragmentDataset(data)
        train_loader = DataLoader(fragment_dataset, batch_size=batch_size, shuffle=True)

        model.train()
        for x, _ in train_loader:
            # x is of shape (batch_size, 1, 30, 11)
            outputs = model(x)
            x_hat = outputs['x_hat']
            loss = loss_function(x_hat, x)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.item())

    avg_loss = np.mean(epoch_loss)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.6f}")

    # Save the model
    torch.save(model.state_dict(), f"autoencoder_epoch_{epoch + 1}.pth")

print("Training complete!")

## Put all on gpu memory

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
import pyarrow.parquet as pq
import numpy as np
import torch.nn.functional as F

class FragmentDataset(Dataset):
    def __init__(self, data):
        self.data = data  # Data is already a GPU tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        x = x.view(1, 30, 11)  # Original shape: (1, 30, 11)
        x = F.pad(x, pad=(0, 5, 0, 2))  # Pad (left, right, top, bottom) to reach (1, 32, 16)
        return x, x  # Input and target for autoencoder

# Directory where the separate Parquet files are stored
fragments_dir = r'X:\Processed_data'

# Model, optimizer, and loss function
num_epochs = 2
batch_size = 512  # Adjust based on GPU memory
model = AutoEncoder(hidden_units1, hidden_units2, hidden_units3, latent_features)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
loss_function = torch.nn.MSELoss()

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)  # Move model to the device

# Iterate through the fragment files
fragment_files = [
    os.path.join(fragments_dir, f)
    for f in os.listdir(fragments_dir)
    if f.endswith('.parquet')
]

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = []

    for file_path in fragment_files:
        print(f"Processing fragment: {file_path}")

        # Load data in main process
        parquet_file = pq.ParquetFile(file_path)
        data = parquet_file.read().to_pandas().to_numpy(dtype='float32')

        # Convert data to tensor and move to GPU
        data_tensor = torch.tensor(data, dtype=torch.float32).to(device)

        # Create Dataset with data already on GPU
        fragment_dataset = FragmentDataset(data_tensor)

        # Since data is on GPU, no need for multiple workers or pin_memory
        train_loader = DataLoader(
            fragment_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0  # Must be 0 when data is on GPU
        )

        model.train()
        for x, _ in train_loader:
            # x is of shape (batch_size, 1, 32, 16)
            outputs = model(x)
            x_hat = outputs['x_hat']
            loss = loss_function(x_hat, x)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.item())

    avg_loss = np.mean(epoch_loss)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.6f}")

    # Save the model
    torch.save(model, f"autoencoder_CNN_epoch_{epoch + 1}.pth")

print("Training complete!")

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the entire model
net_loaded = torch.load('autoencoder_epoch_2.pth', map_location=device)
net_loaded.to(device)
net_loaded.eval()

In [4]:
import os
import torch
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt

# Define the detect_anomalies function with batch processing
def detect_anomalies_in_batches(data, model, device, batch_size=512, threshold=0.08):
    model.eval()
    dataset = torch.utils.data.TensorDataset(torch.tensor(data, dtype=torch.float32))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    reconstruction_errors = []
    
    with torch.no_grad():
        for batch in dataloader:
            data_tensor = batch[0].to(device)
            outputs = model(data_tensor)
            x_hat = outputs['x_hat']
            batch_errors = torch.mean((data_tensor - x_hat) ** 2, dim=1).cpu().numpy()
            reconstruction_errors.extend(batch_errors)
    
    reconstruction_errors = np.array(reconstruction_errors)
    anomalies = reconstruction_errors > threshold
    return anomalies, reconstruction_errors

## Test på træningsdata

In [ ]:
# Directory where the separate Parquet files are stored
fragments_dir = r'X:\Processed_data'

# Iterate through the fragment files
fragment_files = [
    os.path.join(fragments_dir, f)
    for f in os.listdir(fragments_dir)
    if f.endswith('.parquet')
]

file_number = 1

# Ensure that device is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the trained model
# Assuming 'model' is your trained model
# If you have saved the model, load it using:
# model.load_state_dict(torch.load('autoencoder_epoch_10.pth'))
model = AutoEncoder(hidden_units1, hidden_units2, hidden_units3, latent_features)
model.to(device)

# Read the data from the Parquet file
print(fragment_files[file_number])
parquet_file = pq.ParquetFile(fragment_files[file_number])
data_table = parquet_file.read()
data = data_table.to_pandas().to_numpy(dtype='float32')


# Call the detect_anomalies function
anomalies, reconstruction_errors = detect_anomalies_in_batches(data, model, device)

# Plot the reconstruction error distribution
plt.figure(figsize=(10, 6))
plt.hist(reconstruction_errors, bins=50, alpha=0.75)
plt.axvline(x=0.08, color='r', linestyle='--')
plt.title('Reconstruction Error Distribution')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.show()

# Print out the indices of anomalies
anomaly_indices = np.where(anomalies)[0]
print(f"Detected {len(anomaly_indices):,} anomalies out of {len(data):,} samples.")

In [10]:
data_table = pq.read_table(fragment_files[file_number])
data_df = data_table.to_pandas()
data_df['reconstruction_error'] = reconstruction_errors
data_df['anomaly'] = anomalies

In [35]:
#data_df.head(25)

## Test på valideringsdata

In [6]:
freezer_number = "806031"

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'

input_file = freezer_number + "_temp.parquet"  # Replace with the correct file name if needed

# Specify the columns to scale
columns_to_scale = [
    'RTD', '1st Suc.', 'Cond. Air In', 'Evap. In', 'Evap. Out', 
    '2nd Suc.', 'Chil. water In', '2nd Sump', 'H.E.', 
    'SetPoint', 'Mains Voltage'
]

# Load the combined data (assuming it's already saved as 'combined_data.parquet')
data = pd.read_parquet(path + input_file)

# Filter out rows where the freezer is off
data = data[(data['Mains Voltage'] >= 200)]

# Check if all columns are present in the dataset
missing_columns = [col for col in columns_to_scale if col not in data.columns]
if missing_columns:
    raise ValueError(f"The following columns are missing in the data: {missing_columns}")

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Scale the specified columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Save the scaled dataset back to Parquet
scaled_output_file = "scaled_" + freezer_number + "_data.parquet"
data.to_parquet(path + scaled_output_file)

print(f"Scaled data saved as {scaled_output_file}")

In [ ]:
import pandas as pd
import dask.dataframe as dd
import dask.array as da
from dask.delayed import delayed
import os
import tempfile
import shutil
from joblib import Parallel, delayed as joblib_delayed
from tqdm import tqdm
import gc  # Import garbage collection module

##### Brug ML env #####

# Constants
input_path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'
output_path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'
window_size = 30
num_cores = -1  # Use all available cores

# List of freezer numbers to process
freezer_numbers = [ freezer_number ]

# Function to process a single freezer
def process_freezer(freezer_number):
    temp_dir = tempfile.mkdtemp()
    temp_output_file = os.path.join(temp_dir, f"processed_{freezer_number}_temp")
    final_output_file = os.path.join(output_path, f"test_ready_{freezer_number}.parquet")

    try:
        # Load data using Dask
        df = dd.read_parquet(input_path + f'scaled_{freezer_number}_data.parquet')
        
        # Step 1: Drop unnecessary columns
        drop_columns_set_1 = ['State', 'Type', 'Event']
        drop_columns_set_2 = drop_columns_set_1 + ['main_fault']
        if 'main_fault' in df.columns:
            df = df.drop(columns=[col for col in drop_columns_set_2 if col in df.columns])
        else:
            df = df.drop(columns=[col for col in drop_columns_set_1 if col in df.columns])

        # Convert to pandas for time-based operations
        df = df.compute()

        # Step 2: Ensure datetime format
        df['Datetime'] = pd.to_datetime(df['Datetime'])

        # Step 3: Identify continuous sequences
        df['time_diff'] = df['Datetime'].diff().dt.total_seconds()
        df['is_continuous'] = (df['time_diff'].between(1, 120)) | (df.index == 0)
        df['sequence_group'] = (~df['is_continuous']).cumsum()

        # Step 5: Filter valid sequences
        valid_sequences = df.groupby('sequence_group').filter(lambda x: len(x) >= window_size)

        print(valid_sequences.shape)

        datetime_file = "datetime_after_seq_" + freezer_number + ".parquet"
        valid_sequences[['Datetime']].to_parquet(input_path + datetime_file, index=False)

        # Get the original feature names
        feature_names = [col for col in valid_sequences.columns 
                         if col not in ['Datetime', 'time_diff', 'is_continuous', 'sequence_group']]

        # Step 6: Process groups in chunks
        def process_group(group):
            group_windows = []
            for start_idx in range(0, len(group) - window_size + 1):
                window = group.iloc[start_idx:start_idx + window_size]
                flattened_window = window.drop(
                    ['Datetime', 'time_diff', 'is_continuous', 'sequence_group'], axis=1
                ).values.flatten()
                group_windows.append(flattened_window)
            return group_windows

        # Process groups in chunks to manage memory
        chunk_size = 20
        groups = [group for _, group in valid_sequences.groupby('sequence_group')]
        num_chunks = (len(groups) + chunk_size - 1) // chunk_size

        # Create column names for the flattened data
        feature_columns = [f'feature_{i}_{t}' for i in range(window_size) for t in feature_names]

        # Process chunks and write to temporary file
        first_chunk = True

        for chunk_idx in range(num_chunks):
            start_idx = chunk_idx * chunk_size
            end_idx = min((chunk_idx + 1) * chunk_size, len(groups))
            chunk_groups = groups[start_idx:end_idx]
            
            # Process chunk using parallel processing
            chunk_windows = Parallel(n_jobs=num_cores)(
                joblib_delayed(process_group)(group) for group in tqdm(
                    chunk_groups, 
                    desc=f"Processing Freezer {freezer_number} Chunk {chunk_idx + 1}/{num_chunks}",
                    unit="group"
                )
            )
            
            # Flatten the chunk results
            chunk_windows = [window for group_windows in chunk_windows for window in group_windows]
            
            # Convert chunk to DataFrame with proper column names
            chunk_df = pd.DataFrame(chunk_windows, columns=feature_columns)
            
            # Write chunk to temporary parquet file
            if first_chunk:
                chunk_df.to_parquet(temp_output_file, index=False)
                first_chunk = False
            else:
                chunk_df.to_parquet(temp_output_file, index=False, append=True)
            
            # Clear memory
            del chunk_windows
            del chunk_df
            gc.collect()  # Trigger garbage collection

        # Move the temporary file to the final location with a new name
        shutil.move(temp_output_file, final_output_file)
        
        print(f"Successfully saved processed file to: {final_output_file}")

    finally:
        # Clean up temporary directory
        try:
            shutil.rmtree(temp_dir)
        except:
            pass
        
        # Explicitly delete variables and run garbage collection
        del df, valid_sequences, groups, feature_names, drop_columns_set_1, drop_columns_set_2
        gc.collect()

# Process all freezers
for freezer_number in freezer_numbers:
    print(f"Processing freezer: {freezer_number}")
    process_freezer(freezer_number)

In [ ]:
import pandas as pd
import dask.dataframe as dd
import dask.array as da
from dask.delayed import delayed
import os
import tempfile
import shutil
from joblib import Parallel, delayed as joblib_delayed
from tqdm import tqdm
import gc  # Import garbage collection module

##### Brug ML env #####

# Constants
input_path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'
output_path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'
window_size = 30
num_cores = -1  # Use all available cores

# List of freezer numbers to process
freezer_numbers = [ freezer_number ]

# Function to process a single freezer
def process_freezer(freezer_number):
    temp_dir = tempfile.mkdtemp()
    temp_output_file = os.path.join(temp_dir, f"processed_{freezer_number}_temp")
    final_output_file = os.path.join(output_path, f"test_ready_{freezer_number}.parquet")

    try:
        # Load data using Dask
        df = dd.read_parquet(input_path + f'scaled_{freezer_number}_data.parquet')
        
        # Step 1: Drop unnecessary columns
        drop_columns_set_1 = ['State', 'Type', 'Event']
        drop_columns_set_2 = drop_columns_set_1 + ['main_fault']
        if 'main_fault' in df.columns:
            df = df.drop(columns=[col for col in drop_columns_set_2 if col in df.columns])
        else:
            df = df.drop(columns=[col for col in drop_columns_set_1 if col in df.columns])

        # Convert to pandas for time-based operations
        df = df.compute()

        # Step 2: Ensure datetime format
        df['Datetime'] = pd.to_datetime(df['Datetime'])

        # Step 3: Identify continuous sequences
        df['time_diff'] = df['Datetime'].diff().dt.total_seconds()
        df['is_continuous'] = (df['time_diff'].between(1, 120)) | (df.index == 0)
        df['sequence_group'] = (~df['is_continuous']).cumsum()

        # Step 5: Filter valid sequences
        valid_sequences = df.groupby('sequence_group').filter(lambda x: len(x) >= window_size)

        # Get the original feature names
        feature_names = [col for col in valid_sequences.columns 
                         if col not in ['Datetime', 'time_diff', 'is_continuous', 'sequence_group']]

        def process_group(group):
            group_windows = []
            datetime_output = []  # Store one Datetime per flattened window

            for start_idx in range(0, len(group) - window_size + 1):
                window = group.iloc[start_idx:start_idx + window_size]
                
                # Save the first 'Datetime' of the window as representative for the flattened window
                datetime_output.append(window['Datetime'].iloc[0])
                
                # Flatten the other columns (excluding 'Datetime')
                flattened_window = window.drop(
                    ['Datetime', 'time_diff', 'is_continuous', 'sequence_group'], axis=1
                ).values.flatten()
                group_windows.append(flattened_window)
            
            return group_windows, datetime_output

        # Process groups in chunks to manage memory
        chunk_size = 20
        groups = [group for _, group in valid_sequences.groupby('sequence_group')]
        num_chunks = (len(groups) + chunk_size - 1) // chunk_size

        # Create column names for the flattened data
        feature_columns = [f'feature_{i}_{t}' for i in range(window_size) for t in feature_names]

        # Process chunks and write to temporary file
        first_chunk = True

        for chunk_idx in range(num_chunks):
            start_idx = chunk_idx * chunk_size
            end_idx = min((chunk_idx + 1) * chunk_size, len(groups))
            chunk_groups = groups[start_idx:end_idx]
            
            # Process chunk using parallel processing
            chunk_results = Parallel(n_jobs=num_cores)(
                joblib_delayed(process_group)(group) for group in tqdm(
                    chunk_groups, 
                    desc=f"Processing Freezer {freezer_number} Chunk {chunk_idx + 1}/{num_chunks}",
                    unit="group"
                )
            )
            
            # Split the results into data windows and datetime outputs
            chunk_windows, datetime_outputs = zip(*chunk_results)

            # Flatten the chunk results
            chunk_windows = [window for group_windows in chunk_windows for window in group_windows]
            datetime_outputs = [dt for group_datetimes in datetime_outputs for dt in group_datetimes]

            # Convert chunk to DataFrame
            chunk_df = pd.DataFrame(chunk_windows, columns=feature_columns)
            chunk_df['Datetime'] = datetime_outputs  # Add Datetime column to the output DataFrame

            # Write processed data to temporary parquet file
            if first_chunk:
                chunk_df.to_parquet(temp_output_file, index=False)
                first_chunk = False
            else:
                chunk_df.to_parquet(temp_output_file, index=False, append=True)
            
            # Clear memory
            del chunk_windows, datetime_outputs, chunk_df
            gc.collect()

        # Move the temporary file to the final location with a new name
        shutil.move(temp_output_file, final_output_file)
        
        print(f"Successfully saved processed file to: {final_output_file}")

    finally:
        # Clean up temporary directory
        try:
            shutil.rmtree(temp_dir)
        except:
            pass
        
        # Explicitly delete variables and run garbage collection
        del df, valid_sequences, groups, feature_names, drop_columns_set_1, drop_columns_set_2
        gc.collect()

# Process all freezers
for freezer_number in freezer_numbers:
    print(f"Processing freezer: {freezer_number}")
    process_freezer(freezer_number)

In [ ]:
path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'


#data_df = pd.read_parquet(path + "806031_temp.parquet")
#data_df = pd.read_parquet(path + "scaled_806031_data.parquet")
data_df = pd.read_parquet(path + "test_ready_806031.parquet")

path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'

data_df = pd.read_parquet(path + "test_ready_" + freezer_number + ".parquet")

data_df = data_df.drop(['Datetime'], axis=1)

data_df.head()

In [ ]:
path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'

data_df = pd.read_parquet(path + "test_ready_" + freezer_number + ".parquet")

data_df = data_df.drop(columns=['Datetime'])

# Ensure that device is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoEncoder(hidden_units1, hidden_units2, hidden_units3, latent_features)
model.to(device)

data = data_df.to_numpy(dtype='float32')

# Call the detect_anomalies function
anomalies, reconstruction_errors = detect_anomalies_in_batches(data, model, device)

# Plot the reconstruction error distribution
plt.figure(figsize=(10, 6))
plt.hist(reconstruction_errors, bins=50, alpha=0.75)
plt.axvline(x=0.08, color='r', linestyle='--')
plt.title('Reconstruction Error Distribution')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.show()

# Print out the indices of anomalies
anomaly_indices = np.where(anomalies)[0]
print(f"Detected {len(anomaly_indices):,} anomalies out of {len(data):,} samples.")

In [ ]:
path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'

data_df = pd.read_parquet(path + "test_ready_" + freezer_number + ".parquet")

data_df = data_df.drop(columns=['Datetime'])

# Ensure that device is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoEncoder(hidden_units1, hidden_units2, hidden_units3, latent_features)
model.to(device)

data = data_df.to_numpy(dtype='float32')

# Call the detect_anomalies function
anomalies, reconstruction_errors = detect_anomalies_in_batches(data, model, device)

data_df = pd.read_parquet(path + "test_ready_" + freezer_number + ".parquet")

data_df['reconstruction_error'] = reconstruction_errors
data_df['anomaly'] = anomalies

In [ ]:
path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'

data_df = pd.read_parquet(path + "test_ready_" + freezer_number + ".parquet")

data_df['reconstruction_error'] = reconstruction_errors
data_df['anomaly'] = anomalies

# Extract anomaly timestamps from data_df
anomaly_timestamps = data_df['Datetime'][data_df['anomaly']].values

# Downsample the data for faster plotting (e.g., every 10th point)
downsample_rate = 10
downsampled_data = data_df.iloc[::downsample_rate]
downsampled_anomaly_timestamps = anomaly_timestamps[::downsample_rate]

# Plot the first column over time
plt.figure(figsize=(15, 6))
plt.plot(downsampled_data['Datetime'], downsampled_data['feature_0_RTD'], label='feature_0_RTD')

# Add vertical red lines for anomalies
for ts in anomaly_timestamps:
    plt.axvline(x=ts, color='r', linestyle='--', alpha=0.1)

plt.title(' Over Time with Detected Anomalies')
plt.xlabel('Datetime')
plt.ylabel('RTD')
plt.legend(loc='upper right')  # Choose a fixed location for the legend
plt.tight_layout()
plt.show()